# Radiomics Feature Extraction

Extracting features from PET and CT images.

In [1]:
import nrrd
import utils 
import radiomics

import numpy as np
import pandas as pd

from feature_extraction import feature_extractor

In [2]:
# Path to images.
path_ct_imagedir = './../../../data_source/images/ct_nrrd/'
path_pet_imagedir = './../../../data_source/images/pet_nrrd/'
path_masksdir = './../../../data_source/images/masks_nrrd/'

In [3]:
paths_ct_images = utils.sample_paths(
    path_ct_imagedir, path_masksdir, target_format='nrrd'
)
paths_pet_images = utils.sample_paths(
    path_pet_imagedir, path_masksdir, target_format='nrrd'
)

## Setup

In [4]:
def bin_widths(path_images, nbins, n=3):
    """Average min and max to compare histogram across stacks.
    Fixed bin iwdth (and not fixed bin size) to compare texture
    features across stacks.
    
    """
    img_max, img_min = [], []
    for image_path in path_images:
        image, _ = nrrd.read(image_path['Image'])
        # Z-score transformation.
        image_std = np.std(image)
        image_zscore = (image - np.mean(image)) / image_std
        # Remove outliers.
        outliers = n * image_std 
        image_zscore = np.clip(image_zscore, -1.0 * outliers, outliers)
        
        img_max.append(np.ceil(np.max(image_zscore)))
        img_min.append(np.floor(np.min(image_zscore)))
  
    return (np.mean(img_max) - np.mean(img_min)) / nbins

In [5]:
bin_widths(paths_ct_images, 32), bin_widths(paths_ct_images, 64), bin_widths(paths_ct_images, 128)

(0.30950126262626265, 0.15475063131313133, 0.07737531565656566)

In [6]:
bin_widths(paths_pet_images, 32), bin_widths(paths_pet_images, 64), bin_widths(paths_pet_images, 128)

(0.2032828282828283, 0.10164141414141414, 0.05082070707070707)

## Calculate features

In [7]:
ct_param_files = [
    './parameter_files/no_filter/ct_32_no_filter_config.yaml',
    './parameter_files/no_filter/ct_64_no_filter_config.yaml',
    './parameter_files/no_filter/ct_128_no_filter_config.yaml',
]
ct_feature_files = [
    './../../../data_source/radiomic_features/no_filter/ct_32_no_filter_config.csv',
    './../../../data_source/radiomic_features/no_filter/ct_64_no_filter_config.csv',
    './../../../data_source/radiomic_features/no_filter/ct_128_no_filter_config.csv',
]
for ct_param_file, ct_feature_file in zip(ct_param_files, ct_feature_files):
    feature_extractor(
        ct_param_file,
        paths_ct_images, 
        verbose=1, 
        path_to_results=ct_feature_file
    )

Initiated feature extraction.


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    9.7s
[Parallel(n_jobs=3)]: Done 198 out of 198 | elapsed:   37.2s finished


Initiated feature extraction.


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    8.4s
[Parallel(n_jobs=3)]: Done 198 out of 198 | elapsed:   29.9s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Initiated feature extraction.


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:   10.3s
[Parallel(n_jobs=3)]: Done 198 out of 198 | elapsed:   40.2s finished


In [8]:
pet_param_files = [
    './parameter_files/no_filter/pet_32_no_filter_config.yaml',
    './parameter_files/no_filter/pet_64_no_filter_config.yaml',
    './parameter_files/no_filter/pet_128_no_filter_config.yaml'
]
pet_feature_files = [
    './../../../data_source/radiomic_features/no_filter/pet_32_no_filter_config.csv',
    './../../../data_source/radiomic_features/no_filter/pet_64_no_filter_config.csv',
    './../../../data_source/radiomic_features/no_filter/pet_128_no_filter_config.csv'
]
for pet_param_file, pet_feature_file in zip(pet_param_files, pet_feature_files):
    feature_extractor(
        pet_param_file,
        paths_pet_images, 
        verbose=1, 
        path_to_results=pet_feature_file
    )

[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Initiated feature extraction.


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    7.6s
[Parallel(n_jobs=3)]: Done 198 out of 198 | elapsed:   28.4s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Initiated feature extraction.


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:    7.2s
[Parallel(n_jobs=3)]: Done 198 out of 198 | elapsed:   28.0s finished
[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


Initiated feature extraction.


[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed:   10.6s
[Parallel(n_jobs=3)]: Done 198 out of 198 | elapsed:   44.3s finished
